In [3]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx
import matplotlib.colors as colors
import pandas as pd
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from scipy.spatial.distance import cosine
from scipy.cluster.vq import vq,kmeans,whiten
import time
%matplotlib inline



In [5]:
#functions 
#input: product ID
#output: sparse vector based on vectorizer

def to_vector(product_id):
    temp = reviews[reviews.ProductId == product_id]
    text = list(temp['Text'].values + ' ' + temp['Summary'].values)
    new_string = np.array([''.join(text)]).reshape(1,)
    return vectorizer.transform(new_string).toarray()

def to_words(product_id):
    temp = reviews[reviews.ProductId == product_id]
    text = list(temp['Text'].astype(str).values + ' ' + temp['Summary'].astype(str).values)
    return np.array([''.join(text)]).reshape(1,)



Get Unique products and users

In [2]:
reviews = pd.read_csv('Reviews.csv')
unique_users = reviews.UserId.unique()
unique_products = reviews.ProductId.unique()

In [6]:
#whole thing takes a long time(>30minutes), use first 1000 products
start = time.time()
unique_prod_reviews = map(lambda x: to_words(x),unique_products[0:1000])
end = time.time()
print end - start

29.6910879612


In [7]:
#Show the first 5 entries of the product DF
product_df = pd.DataFrame()
product_df['ProductId'] = unique_products[0:1000]
product_df['Text'] = unique_prod_reviews
product_df.head()

,ProductId,Text
0,B001E4KFG0,[I have bought several of the Vitality canned ...
1,B00813GRG4,[Product arrived labeled as Jumbo Salted Peanu...
2,B000LQOCH0,[This is a confection that has been around a f...
3,B000UA0QIQ,[If you are looking for the secret ingredient ...
4,B006K2ZZ7K,[Great taffy at a great price. There was a wi...


In [8]:
a = np.array(unique_prod_reviews)
a = a.reshape(len(a),)
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(a)

In [9]:
tfidf_matrix.toarray()
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_similarities

array([[  1.00000000e+00,   5.36110174e-03,   0.00000000e+00, ...,
          8.28603989e-03,   5.19533133e-03,   5.02833995e-03],
       [  5.36110174e-03,   1.00000000e+00,   0.00000000e+00, ...,
          5.29899866e-03,   0.00000000e+00,   1.78627910e-03],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  8.28603989e-03,   5.29899866e-03,   0.00000000e+00, ...,
          1.00000000e+00,   8.30201976e-03,   7.89411311e-04],
       [  5.19533133e-03,   0.00000000e+00,   0.00000000e+00, ...,
          8.30201976e-03,   1.00000000e+00,   4.70532436e-03],
       [  5.02833995e-03,   1.78627910e-03,   0.00000000e+00, ...,
          7.89411311e-04,   4.70532436e-03,   1.00000000e+00]])

In [10]:
#takes a product name in the unique products data and returns the 8 most similar products
#input: Product ID, Product Dataframe, Similarity Matrix
#output: Array of 8 nearest items in the database using cosine similarity

def return_nearest(product,cosine_similarities,product_df):
    idx = product_df[product_df.ProductId == product].index[0]
    similar_indices = cosine_similarities[idx].argsort()[:-10:-1]
    similar_items = [(cosine_similarities[idx][i], product_df['ProductId'][i]) for i in similar_indices]
    return similar_items[1:]

In [11]:
return_nearest('B00813GRG4',cosine_similarities,product_df)

[(0.060071873611170931, 'B000CMFMG8'),
 (0.052140838382313696, 'B0047LRBX2'),
 (0.043432096349825311, 'B001EO6BCC'),
 (0.037325003722012799, 'B000G6RPMY'),
 (0.026276191344016604, 'B001FA1L7U'),
 (0.025318462964514555, 'B004AVYUOW'),
 (0.025057186536542934, 'B001HOUGFC'),
 (0.023818763987623219, 'B000YT5DBS')]

In [12]:
def number_reviews(users):
    number_reviews_ = []
    for user in users:
        number_reviews_.append(reviews[reviews['UserId'] == user].values.shape[0])
    return np.array(number_reviews_)
        
start = time.time()    
number_reviews_1000 = number_reviews(unique_users[0:1000])
end = time.time()

In [13]:
#lets consolidate everything we're working with
u_users = unique_users[0:1000]
print number_reviews_1000

[  1   1   3   3   2   1   2   1   2   4   4  12   2  20 201   1   2   1
  49   2   1   2   2   4   2   3   2   1   1   1   2  17   4   1   4   3
   1   2   6  10   1   1   6  88  12 133   1  14   3   2   2   9   2   2
   2   1   1   1   3   1   1   1   1   1   1   1   2   2   3   5   1   3
  21   1   4  13  16   1  17   1   1   3   2   4   1   1   2   1   1   1
   1   1   2   1   1   9   1   1   1  12   8  18   8  11   7   7   2   4
 389   2   4   4   4   4   4   1   1   8   5   5   4   4   9   4   6   4
   1  10   1   1   1   3   3  15   3  10   1   1   1   1  75   6 256  27
   1   1   1   2   4   2   1   1   1   5   1   5   3  47   4   1   1   1
  15   1   4   1   1   3   3   3   1   2   1   2   9   7  33  23   1  22
   1   7   1   3   1 140   1   1   1   1  49   1   3   8   1   2   1   1
   2   1   3   1   1   1   1   1   1   9   1   1  10   2   8   1   1   3
  59   1   2   1   3   2   1   2   9   1   1   1   1   1  99   1   1   4
   8   1   6  42  10   1   2   1   1   2   5   1   

We see that there is a large population of user that write a large number of reviews. For our recommender system we can focus on the subset of the population that writes greater than $N$ reviews, so that there is more data upon which to base a recommender model. 

### Constructing a Rating Matrix
- Define 0 to be unrated
- 1 through 5 as usual

We will focus on our subset of 1000 users but look at all products

In [15]:
len(unique_products)

74258

In [16]:
len(unique_users)

256059

In [33]:
reviews[reviews['UserId']==u_users[100]][['ProductId','Score']].values

array([['B004K2IHUO', 4],
       ['B006L4FNB0', 4],
       ['B00115FPV2', 5],
       ['B009AVD282', 4],
       ['B002Y1CVZU', 4],
       ['B00815FR68', 4],
       ['B000PKMN14', 4],
       ['B006L4FMR0', 4]], dtype=object)

In [34]:
def construct_rating_matrix(u_users,unique_products):
    r_matrix = np.zeros((len(u_users),len(unique_products)))
    for i in range(len(u_users)):
        user = u_users[i]
        products = reviews[reviews['UserId']== user][['ProductId','Score']].values
        for j in range(len(products)):
            product = products[j][0]
            stars = products[j][1]
            index = np.argwhere(unique_products == product)
            r_matrix[i,index] = stars
    return r_matrix 
            
rating_matrix = construct_rating_matrix(u_users,unique_products)        

YAY!

Generating predictions for a user $u$ requires using $s: U \times U \to R$, similarity function, to compute a neighborhood $ N \subseteq U$ of neighbors of $U$

#### Possible User/Product Similarity Functions:
- Pearson Correlation: compute the statistical correlation between two vectors. This is the correlation as it is usually defined, sample covariance divided by the standard deviations.

- Cosine similarity: similarity is measured as the cosine distance between two rating vectors. 
$$ \frac{r_{u} \cdot r_{v}}{||r_{u}|||r_{v}||}$$


#### Other Considerations: 
- Subtraction off the mean of each user to account for differences in users' use of the rating scale
- Tuning parameter: number of group neighbors to select

### How do we evaluate recommender systems?
- We want to look at users that write a lot of reviews, so that we can validate our model's predictions by dividing their reviews into two parts
- Training/Test set: We build the recommender system upon the training set. For the users in the test set we divide into **query set** and **target set**. 
- The **query set** is fed into the recommender system and the function we want to maximize is the similarity score between the recommendations based on the query set and the **target set**